# Serialization semantics
This note describes how you can save and load PyTorch tensors and module state.

In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <tuple>
#include <string>
#include <vector>
#include <torch/torch.h>
#include <torch/script.h>
namespace nn = torch::nn;

## Saving and loading tensors
torch.save() and torch.load() let you easily save and load tensors:

In [3]:
torch::Tensor t = torch::tensor({1., 2.});
std::cout << t << std::endl;

 1
 2
[ CPUFloatType{2} ]


In [4]:
torch::save(t, "./tensor_cxx.pt");

In [5]:
torch::Tensor loaded_t;
torch::load(loaded_t, "./tensor_cxx.pt");

In [6]:
std::cout << loaded_t << std::endl;

 1
 2
[ CPUFloatType{2} ]


## Saving and loading torch.nn.Modules

In [7]:
auto options = nn::BatchNorm1dOptions(/*num_features*/4).eps(0.0).momentum(1.0).affine(true).track_running_stats(true);

In [8]:
nn::BatchNorm1d  batch_norm_1d(options);

https://github.com/pytorch/pytorch/blob/master/torch/csrc/api/include/torch/nn/module.h

https://github.com/pytorch/pytorch/blob/master/torch/csrc/api/include/torch/ordered_dict.h

In [9]:
torch::OrderedDict<std::string, torch::Tensor> ordered_parameter_dict = batch_norm_1d ->named_parameters();

In [10]:
std::cout << ordered_parameter_dict.key_description();

Key

In [11]:
std::cout << ordered_parameter_dict["weight"];

 1
 1
 1
 1
[ CPUFloatType{4} ]

In [12]:
std::cout << ordered_parameter_dict["bias"];

 0
 0
 0
 0
[ CPUFloatType{4} ]

In [13]:
// can not to save ordered_dict
//torch::save(ordered_parameter_dict, "ordered_parameter_dict.pt");

In [14]:
torch::OrderedDict<std::string, torch::Tensor> ordered_buffer_dict = batch_norm_1d ->named_buffers();

In [15]:
std::cout << ordered_buffer_dict["running_mean"];

 0
 0
 0
 0
[ CPUFloatType{4} ]

In [16]:
torch::save(batch_norm_1d, "batch_norm_1d.pt");

In [17]:
// load the model
torch::load(batch_norm_1d, "batch_norm_1d.pt");

In [18]:
auto model =  batch_norm_1d;

for(auto& p: model->named_parameters()){
        //access key
        std::cout << p.key() << std::endl;
    
        // access value
        std::cout << p.value() << std::endl;
    }

weight
 1
 1
 1
 1
[ CPUFloatType{4} ]
bias
 0
 0
 0
 0
[ CPUFloatType{4} ]


## load tensor saved from python

In [19]:
try
{
    torch::jit::script::Module module = torch::jit::load("x.pth");
    c10::IValue iv = module.attr("x");
    torch::Tensor ts = iv.toTensor();
    std::cout << ts;
}catch (const c10::Error& e) {
    std::cerr << "error loading the tensor " << std::endl;
    std::cerr << e.msg() << std::endl;
}

 1  1  1  1
 1  1  1  1
 1  1  1  1
 1  1  1  1
[ CPUFloatType{4,4} ]

## load Module saved from python

In [25]:
try{
    torch::jit::script::Module module;
    module = torch::jit::load("mymodule.pt");
    
    //pprepare input
    std::vector<torch::jit::IValue> input_vector;
    input_vector.push_back(torch::ones({2,4}));
    
    // execute the model and turn its output into a tensor
    at::Tensor output  = module.forward(input_vector).toTensor();
    std::cout << output << std::endl;
    
} catch (const c10::Error& e){
    std::cerr << "error loading the module " << std::endl;
    std::cerr << e.msg() << std::endl;
}

 0.1472
 0.1472
[ CPUFloatType{2,1} ]
